In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import matplotlib.patches as mpatches

from matplotlib.collections import PatchCollection
import matplotlib

In [ ]:
#list of number of link that are assigned
linkSummary = pd.read_csv('geomInfo/ModuleLinkSummary.csv')

# six cassettes are put together to form one layer. Currently, we look at only 0th cassete of a few layers (5,7,9)
linkSummary = linkSummary[(linkSummary.Cassette==0) & (linkSummary.Layer>=5) & (linkSummary.Layer<=9)]

# The (ModU, modV) is the (U, V) coordinate of a module (=wafer=econ) within a layer
linkSummary.set_index(['Layer','ModU','ModV'],inplace=True)
linkSummary

In [ ]:
#load data from csv files.  This is used to extract average data sizes
#daq_Data = pd.concat([pd.read_csv(f'Data/ttbar_copy_new.csv') for i in range(16)])
daq_Data = pd.read_csv(f'Data/ttbar_copy_new.csv')
'''
The daq_Data consist of the entries after applying ZS corresponding to different cells 
(a layer has many wafers, a wafer has many cells). The description about the each of columns
(c1): It is the index of df
(c2 = entry): those events which passed the ZS
(c3 = layer): layer number (from 5 to 9)
(c4 = waferu): U coordinate of the wafer
(c5 = waferv): V coordinate of the wafer

(c6 = HDM): high density module: One of the 8-inch wafers/modules with 432 cells 
(compared to the LDM, or low-density module, which has 192 cells)
the type 0 are 120 micron thick and HDM and closest to beam pipe,
type 1 is 200 microns thick LDM modules, and a little further out
type 2 are 300 micron LDM modules, and furthest from beam pipe in eta
If HDM is True then it correspond to type 0 else 1 or 2

(c7 = occ): Occupancy: Each wafer has 3-6 eLink, each eLink has up to 37 channels, 
we’ve applied the zero suppression, so any channel below the threshold is removed, 
so the count is just counting how many cells on a given eLink on a given wafer pass the ZS

(c8 = eRxPacket_Words): It is the size of the packet in terms of number of words (1 word is of 32 bit).
It is calculated as int(Bits/32+1) + 2, where Bits = 16  + 8* (charge_BX1 + toa_BX2)

(c9  = NonEmptyLinks): We have total 6 (12) eLinks for HDM(LDM). 

(c10 = EmptyLinks): The rest eLinks

(c11 = TotalWords): evt_headerWords (=2) + eRxPacket_Words + EmptyLinks + evt_trailerWords (=2)

''' 
daq_Data

In [ ]:
daq_Data.groupby(['layer','waferu','waferv']).any()

In [ ]:
daq_Data.groupby(['layer','waferu','waferv']).mean()

In [ ]:
#get list of wedges, and assign x and y positions for drawing maps
wedges = daq_Data.groupby(['layer','waferu','waferv']).any()[['HDM']].reset_index()
wedges['y'] = wedges.waferv
wedges['x'] = 0
#QQQQ: why 2+v-2*u?
wedges.loc[:,'x'] = (2+wedges.y-2*wedges.waferu)
#QQQQ: why rescale y and x?
wedges.y *= 1.5
wedges.x *= -3**.5/2
wedges.set_index(['layer','waferu','waferv'],inplace=True)
wedges

In [ ]:
#function to look through output logs of condor jobs to extract and accumulate the results across several runs
# if you just ran interactivel, need to just get the info on the overflows, max size, etc into a dataframe.

def getDF(inputDF, LogName=None,Nstart=0,N=100):
    #QQQQ: Why mean?
    fullData = inputDF.groupby(['layer','waferu','waferv']).mean()
    
    #linkSummary has only 153 rows which means the last 10 rows will be assigned Nan. 
    #QQQQ: Why linkSummary has only 153 rows?
    fullData['eTx_assigned'] = linkSummary.ECOND_eTX
    
    #Replace Nan by 1
    fullData.eTx_assigned = fullData.eTx_assigned.fillna(1).astype(int)
    
    #The numpy.ceil() is a mathematical function that returns the ceil of the elements of array. 
    #The ceil of the scalar x is the smallest integer i, such that i >= x
    #QQQQ: why divide by 53.3333? One eTx can process 53.3333 Words of data?
    fullData['eTx_30percent'] = np.ceil((fullData.TotalWords/53.3333)*1.3).astype(int)
    fullData['eTx_Mean'] = np.ceil((fullData.TotalWords/53.3333))
    '''
    Every condor output has information for 6 econs (they have different # eLinks)
    We collect information from ALL condor outputs for ALL econs in a 2D (163,6) array

    overflows: An econ with few number of eLink is overflow for a few thousands times in 
    few million BX which means for few billion BXs (many condor outputs) the overflows should be added

    maxSize: Among all condor outouts, it is the maximum buffer size. Which means it is NOT additive

    L1As issued: How many times an L1A is issued for a given number of BXs. Which means it is additive

    '''
    if not LogName is None:
        data_overflow = []
        data_maxSize = []
        L1As_issued=0
        
        #Get the 0th element of the list from 0th condor file
        fileName = f'log_100/{LogName}_{Nstart}.stdout'
        with open(fileName,'r') as _file:
            # we loop over all lines of the condor output and extract
            for line in _file:
                if 'overflows= ' in line:
                    #overflows= [...]
                    data_overflow.append(eval(line[10:]))
                if 'maxSize= ' in line:
                    #maxSize= [...]
                    data_maxSize.append(eval(line[8:]))
                if 'L1As issued' in line:
                    #749401 L1As issued
                    L1As_issued += int(line.split()[0])
        #2D array            
        data_overflow=np.array(data_overflow)
        data_maxSize=np.array(data_maxSize)

        for i_file in range(Nstart+1,Nstart+N):
            fileName = f'log_100/{LogName}_{i_file}.stdout'
            with open(fileName,'r') as _file:
                for line in _file:
                    if 'eTx' in line:
                        #1 eTx
                        i = int(line.split()[0])-1
                    if 'overflows= ' in line:
                        overflow=np.array(eval(line[10:]))
                        overflow[overflow<0] = 99999999
                        data_overflow[i] += overflow
                    if 'maxSize= ' in line:
                        maxSize=np.array(eval(line[8:]))
                        data_maxSize[i] = np.maximum(data_maxSize[i], maxSize)
                    if 'L1As issued' in line:
                        L1As_issued += int(line.split()[0])
        
        '''
        The idea is to check how many eLinks are needed for a given wafer in order to avoid overflow.
        For example, the wafer closer to beam axis need more eLinks and vice versa. In this way we can
        efficiantly allocate different number of eLinks to different wafers
        '''
        #transpose (163 col, 6 rows) to (6 cols, 163 rows)
        #In every row look at zero (True==0) and non-zero entries
        #Multiply the respective entry by array([1, 2, 3, 4, 5, 6])
        x = ((data_overflow.transpose()==0)*np.arange(1,7))

        #change all zero entries to 99 (we can choose any large value)
        x[x==0]=99
        
        #for every row, get the minimum value (out of 6 econs, pick the one which has minimum eLink)
        minLinks = x.min(axis=1)
        fullData['min_eTx'] = minLinks
        nLinks = minLinks
        
        #Get the maximum size of the buffer of the corresponding econ with minLinks
        maxSize = []
        for i in range(len(nLinks)):
            maxSize.append(data_maxSize.transpose()[i][nLinks[i]-1])
        fullData['maxSize'] = maxSize    

        
        nLinks = fullData.eTx_assigned.values
        maxSize = []
        overflows = []
        for i in range(len(nLinks)):
            maxSize.append(data_maxSize.transpose()[i][nLinks[i]-1])
            overflows.append(data_overflow.transpose()[i][nLinks[i]-1])
        fullData['maxSize_assigned'] = maxSize
        fullData['overflows_assigned'] = overflows


        nLinks = fullData.eTx_30percent.values
        maxSize = []
        overflows = []
        for i in range(len(nLinks)):
            maxSize.append(data_maxSize.transpose()[i][nLinks[i]-1])
            overflows.append(data_overflow.transpose()[i][nLinks[i]-1])
        fullData['maxSize_30percent'] = maxSize
        fullData['overflows_30percent'] = overflows
        print("fullData = ", fullData)
        #print("L1As_issued = ", L1As_issued)
        #print("data_overflow = ", data_overflow)
        #print("data_maxSize = ", data_maxSize)
    return fullData, L1As_issued, data_overflow, data_maxSize

In [ ]:
fullData,L1As,OF, MS = getDF(daq_Data,LogName='bufferSim__34530053',Nstart=0,N=10)


In [ ]:
fileName = f'log_buffHist.stdout'
with open(fileName,'r') as _file:
    for line in _file:
        if 'buffHist= ' in line:
            #overflows= [...]
            buffHist = eval(line[9:])
#print(buffHist)

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
count = 1
plt.subplots(figsize=(15, 10))
#for h in range(len(buffHist)):
for h in range(20):
    #print(count)
    plt.subplot(4, 5, count)
    plt.hist(buffHist[h], bins=20, range=[0.0, 1550], label="wafer=%i"%h)
    plt.legend(loc='upper right')
    plt.yscale('log')
    count+=1
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(OF[0], bins=70, alpha=0.5, label="1 eTx", fill=False, linewidth=3, edgecolor='c',range=[0.0, 0.7e7])
plt.hist(OF[1], bins=70, alpha=0.5, label="2 eTx", fill=False, linewidth=3, edgecolor='b',range=[0.0, 0.7e7])
plt.hist(OF[2], bins=70, alpha=0.5, label="3 eTx", fill=False, linewidth=3, edgecolor='g',range=[0.0, 0.7e7])
plt.hist(OF[3], bins=70, alpha=0.5, label="4 eTx", fill=False, linewidth=3, edgecolor='r',range=[0.0, 0.7e7])
plt.hist(OF[4], bins=70, alpha=0.5, label="5 eTx", fill=False, linewidth=3, edgecolor='m',range=[0.0, 0.7e7])
plt.hist(OF[5], bins=70, alpha=0.5, label="6 eTx", fill=False, linewidth=3, edgecolor='y',range=[0.0, 0.7e7])
plt.xlabel("Overflows", size=14)
plt.ylabel("Number of Wafers", size=14)
plt.yscale('log')
plt.title("Overflows for different numbers of eTx")
plt.legend(loc='upper right')

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(MS[0], bins=20, alpha=0.5, label="1 eTx", fill=False, linewidth=3, edgecolor='c',range=[0.0, 2000])
plt.hist(MS[1], bins=20, alpha=0.5, label="2 eTx", fill=False, linewidth=3, edgecolor='b',range=[0.0, 2000])
plt.hist(MS[2], bins=20, alpha=0.5, label="3 eTx", fill=False, linewidth=3, edgecolor='g',range=[0.0, 2000])
plt.hist(MS[3], bins=20, alpha=0.5, label="4 eTx", fill=False, linewidth=3, edgecolor='r',range=[0.0, 2000])
plt.hist(MS[4], bins=20, alpha=0.5, label="5 eTx", fill=False, linewidth=3, edgecolor='m',range=[0.0, 2000])
plt.hist(MS[5], bins=20, alpha=0.5, label="6 eTx", fill=False, linewidth=3, edgecolor='y',range=[0.0, 2000])
plt.xlabel("Max buffer size", size=14)
plt.ylabel("Number of Wafers", size=14)
plt.yscale('log')
plt.title("Max buffer size for different numbers of eTx")
plt.legend(loc='upper right')

In [ ]:
#dictionaly of the labels for different types of plots
Labels ={'occ':{'Title':'Average L1A Occupancy, Layer %i',
                'colorLabel':'Avg. Occ. Above Zero Suppression',
                'nDec':1,
                'zMax':260},
         'TotalWords':{'Title':'Average L1A Size (32b Words), Layer %i',
                      'colorLabel':'Avg. L1A Size (32b words)',
                      'nDec':1,
                      'zMax':200},
         'EmptyLinks':{'Title':'Average Number of Empty eRx Packets, Layer %i',
                      'colorLabel':'Avg. # of Empty eRx Packets',
                      'nDec':1,
                      'zMax':6},
         'eTx_assigned':{'Title':'Number of eTx assigned, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':0,
                      'zMax':6},
         'eTx_30percent':{'Title':'Number of eTx assuming 30 percent overhead, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':0,
                      'zMax':6},
         'eTx_Mean':{'Title':'Number of eTx based on mean data, no rounding, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':2,
                      'zMax':6},
         'min_eTx':{'Title':'Min number of eTx with 0 overflows, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':0,
                      'zMax':6},
         'maxSize':{'Title':'Max buffer size, with 0 overflows, Layer %i',
                      'colorLabel':'Buffer size (32b words)',
                      'nDec':0,
                      'zMax':1600},
         'maxSize_assigned':{'Title':'Max buffer size, using assigned number of links, Layer %i',
                      'colorLabel':'Buffer size (32b words)',
                      'nDec':0,
                      'zMax':1600},
         'overflows_assigned':{'Title':'Overflow counter, using assigned number of links, Layer %i',
                      'colorLabel':'# of buffer overflows',
                      'nDec':0,
                      'zMax':40000},
         'maxSize_30percent':{'Title':'Max buffer size, using link assignment from 30 percent overhead, Layer %i',
                      'colorLabel':'Buffer size (32b words)',
                      'nDec':0,
                      'zMax':1600},
         'overflows_30percent':{'Title':'Overflow counter, using link assignment from 30 percent overhead, Layer %i',
                      'colorLabel':'# of buffer overflows',
                      'nDec':0,
                      'zMax':40000},
        }

In [ ]:
def makePlot(var,patches,df=fullData,extra=""):
    data = df.loc[layer,var].values.flatten()

    waferCollection = PatchCollection(patches,cmap=matplotlib.cm.coolwarm)
    waferCollection.set_array(data)
    waferCollection.set_clim([0,Labels[var]['zMax']])
    fig, ax = plt.subplots(figsize=(15,10))
    ax.add_collection(waferCollection)
    plt.axis([0,16,-1,15])
    plt.rcParams["figure.autolayout"] = True
    # ax.set_xticklabels([])
    # ax.set_yticklabels([]);
    cbar = plt.colorbar(waferCollection)
    cbar.set_label(Labels[var]['colorLabel'],fontsize=14)

    wedge['data'] = data

    plt.title(Labels[var]['Title']%layer,fontsize=24)
    for x,y,d in wedge[['x','y','data']].values:
    #     s = data[i]
        plt.text(x,y,f'%.{Labels[var]["nDec"]}f'%d,fontsize=14,horizontalalignment='center',verticalalignment='center')

                 

    fig.savefig(f'Plots/{var}_layer{layer}{extra}.png')

In [ ]:
#for layer in [5,7,9]:
for layer in [5]:
    wedge = wedges.loc[layer]
    patches = []
    for w in wedge.index:
        patches.append(mpatches.RegularPolygon((wedge.loc[w].x,wedge.loc[w].y),6, .95))
    for k in Labels:
        print(k)
        makePlot(k,df=fullData,extra="_OldTTbarTests",patches=patches)